# Import Libs

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.optim import Adam, lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import models, transforms
from torch.utils.tensorboard import SummaryWriter

from PIL import Image
import pandas as pd
import numpy  as np
import time
import os

# Configuration

In [14]:
classes = [
    "angry", 
    "disgust", 
    "fear", 
    "happy", 
    "sad", 
    "surprise", 
    "neutral"
]

In [15]:
class Config:
    # dataset
    TRAIN_DS_PATH = './dataset/train.csv'
    VAL_DS_PATH = './dataset/val.csv'
    TEST_DS_PATH = './dataset/finaltest.csv'
    
    # images dir
    TRAIN_IMG_DIR = './dataset/train/'
    VAL_IMG_DIR  = './dataset/val/'
    TEST_IMG_DIR  = './dataset/finaltest/'
    
    # training hyperparams
    EPOCHS = 1
    LR = 1e-4
    BATCH_SIZE = 8
    NUM_WORKERS = 0
    SHUFFLE = True
    
    # saved model path
    MODEL_DIR = './model/'

In [16]:
cfg = Config()

In [17]:
# tensorboard writer
writer = SummaryWriter("runs")

# Dataset

In [18]:
# reference: https://github.com/omarsayed7/Deep-Emotion/blob/master/data_loaders.py
class EmotionDataset(Dataset):
    def __init__(self, csv_file, img_dir, datatype, transform):
        '''
        Pytorch Dataset class
        params:-
                 csv_file : the path of the csv file    (train, validation, test)
                 img_dir  : the directory of the images (train, validation, test)
                 datatype : data type for the dataset   (train, val, test)
                 transform: pytorch transformation over the data
        return :-
                 image, labels
        '''
        self.csv_file = pd.read_csv(csv_file)
        self.labels = self.csv_file['emotion']
        self.img_dir = img_dir
        self.datatype = datatype
        self.transform = transform

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img = Image.open(self.img_dir + self.datatype + str(idx) + '.jpg').convert('RGB')
        labels = np.array(self.labels[idx])
        labels = torch.from_numpy(labels).long()

        if self.transform :
            img = self.transform(img)
        return img, labels

In [19]:
# tranformations
transformation= transforms.Compose([
    transforms.Resize(256),
#     transforms.Grayscale(3), # no need this since you .convert("RGB") at __getitem__
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [20]:
# create datasets
train_dataset = EmotionDataset(
    csv_file=cfg.TRAIN_DS_PATH, 
    img_dir=cfg.TRAIN_IMG_DIR,
    datatype='train',
    transform=transformation
)

validation_dataset = EmotionDataset(
    csv_file=cfg.VAL_DS_PATH, 
    img_dir=cfg.VAL_IMG_DIR,
    datatype='val',
    transform = transformation
)


test_dataset = EmotionDataset(
    csv_file=cfg.TEST_DS_PATH, 
    img_dir=cfg.TEST_IMG_DIR,
    datatype='finaltest',
    transform = transformation
)

In [21]:
# create data loaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=cfg.BATCH_SIZE, 
    shuffle =cfg.SHUFFLE, 
    num_workers=cfg.NUM_WORKERS
)

val_loader = DataLoader(
    validation_dataset, 
    batch_size=cfg.BATCH_SIZE, 
    shuffle =cfg.SHUFFLE, 
    num_workers=cfg.NUM_WORKERS
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=cfg.BATCH_SIZE, 
    shuffle =cfg.SHUFFLE, 
    num_workers=cfg.NUM_WORKERS
)

# Helpers

In [22]:
def train_epoch(model, train_loader, criterion, optimizer, device, steps_print_loss=0):
    """Train the model for 1 epoch
    Args:
        model: nn.Module
        train_loader: train DataLoader
        criterion: callable loss function
        optimizer: pytorch optimizer
        device: torch.device
        steps_print_loss: loss will print out in every specified steps.
    Returns
    -------
    Tuple[Float, Float]
        average train loss and average train accuracy for current epoch
    """
    train_losses = []
    train_corrects = []
    model.train()

    # Iterate over data.
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # prediction
        outputs = model(inputs)

        # calculate loss
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # statistics
        train_losses.append(loss.item())
        train_corrects.append(torch.sum(preds == labels.data).item())
        
        if steps_print_loss and (batch_idx % steps_print_loss == 0):
            print(f"loss: {loss.item():>7f}  [{batch_idx:>5d}/{len(train_loader):>5d}]")
        
    train_loss = sum(train_losses)/len(train_losses)
    train_accuracy = sum(train_corrects)/len(train_loader.dataset)      

    return train_loss, train_accuracy


def val_epoch(model, val_loader, criterion, device):
    """Validate the model for 1 epoch
    Args:
        model: nn.Module
        val_loader: val DataLoader
        criterion: callable loss function
        device: torch.device
    Returns
    -------
    Tuple[Float, Float]
        average val loss and average val accuracy for current epoch
    """

    val_losses = []
    val_corrects = []
    model.eval()

    # Iterate over data
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # prediction
            outputs = model(inputs)

            # calculate loss
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # statistics
            val_losses.append(loss.item())
            val_corrects.append(torch.sum(preds == labels.data).item())
            
        val_loss = sum(val_losses)/len(val_losses)
        val_accuracy = sum(val_corrects)/len(val_loader.dataset)

    return val_loss, val_accuracy

# Training and Evaluating Model

In [23]:
# load pretrained model
model = models.mobilenet.mobilenet_v2(pretrained=True)

In [24]:
# freeze all layers
for param in model.parameters():
    param.requires_grad = False

num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, len(classes))

In [25]:
# transfer to cuda device if any
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Using {device}")

Using cuda


In [26]:
# loss, optimizer and scheduler 
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=cfg.LR)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min')

In [27]:
# Start training and val loops

since = time.time()
for epoch in range(cfg.EPOCHS):
    print(f"Epoch {epoch+1}\n-------------------------------")
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, 500)
    print(f'Average TrainLoss: {train_loss:.4f} \tAverage TrainAcc: {train_acc:.4f}')
    
    # Evaluation
    print("Evaluation:")
    val_loss, val_acc = val_epoch(model, val_loader, criterion, device)
    print(f'Average ValLoss: {val_loss:.4f} \tAverage ValAcc: {val_acc:.4f}')
    
    # log metrics 
    if writer:
        writer.add_scalar("train_loss", train_loss, epoch)
        writer.add_scalar("train_accuracy", train_acc, epoch)
        writer.add_scalar("val_loss", val_loss, epoch)
        writer.add_scalar("val_accuracy", val_acc, epoch)
    
    # schedule lr
    scheduler.step(val_loss)
    print("\n")
        
time_elapsed = time.time() - since
print(f"Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s")

Epoch 1
-------------------------------
loss: 2.027109  [    0/ 3589]
loss: 1.869347  [  500/ 3589]
loss: 1.941542  [ 1000/ 3589]
loss: 1.608768  [ 1500/ 3589]
loss: 1.568424  [ 2000/ 3589]
loss: 1.958847  [ 2500/ 3589]
loss: 1.572384  [ 3000/ 3589]
loss: 1.723830  [ 3500/ 3589]
Average TrainLoss: 1.7177 	Average TrainAcc: 0.3018
Evaluation:
Average ValLoss: 1.6362 	Average ValAcc: 0.3703


Training complete in 2m 38s


# Evaluating on Unseen Test Data

In [29]:
test_losses = []
test_corrects = []

model.eval()
# Iterate over data
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # prediction
        outputs = model(inputs)

        # calculate loss
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # statistics
        test_losses.append(loss.item())
        test_corrects.append(torch.sum(preds == labels.data).item())

    test_loss = sum(test_losses)/len(test_losses)
    test_accuracy = sum(test_corrects)/len(test_loader.dataset)
    
print(f'Average TestLoss: {test_loss:.4f} \tAverage TestAcc: {test_accuracy:.4f}')

Average TestLoss: 1.6329 	Average TestAcc: 0.3525


# Saving model

In [ ]:
# Save the model
model_name = f"model_epoch{cfg.EPOCHS}_lr{cfg.LR}_batch{cfg.BATCH_SIZE}.pt"
torch.save(model.state_dict(), cfg.MODEL_DIR + model_name)